# Training with AMPtorch

Edited by N. Hu
Medford Research Group at Georgia Tech
06/25/2021

This ipython notebook is based on the sample scripts provided by AMPtorch (`amptorch/example/`). It introduces basic components and functions of AMPtorch. 

## Preparation: Install AMPtorch with conda

Please follow the instructions as shown in the github repo to install AMPtorch and its dependencies: <https://github.com/ulissigroup/amptorch/tree/MCSH_paper1_lmdb>

## Training on CuCO with Symmetry Functions and BPNN

This example follows the convention of atomistic neural network potentials as described in Behler, J. (2015). Constructing high-dimensional neural network potentials: A tutorial review. International Journal of Quantum Chemistry, 115(16), 1032–1050. https://doi.org/10.1002/qua.24890

In [15]:
import numpy as np
import torch
from ase import Atoms
from ase.calculators.emt import EMT

from amptorch.ase_utils import AMPtorch
from amptorch.trainer import AtomsTrainer

### Construct the training dataset

As an example, we calculate the potential energy and forces for CuCO chemical system with the EMT calculator implemented in ase. 

This step should be replaced with training datasets that have information on:
1. cell size
2. periodic boundary conditions
3. atomic positions
4. potential energy
5. forces, if doing force training

In [16]:
# get training images

distances = np.linspace(2, 5, 100)
images = []
for dist in distances:
    image = Atoms(
        "CuCO",
        [
            (-dist * np.sin(0.65), dist * np.cos(0.65), 0),
            (0, 0, 0),
            (dist * np.sin(0.65), dist * np.cos(0.65), 0),
        ],
    )
    image.set_cell([10, 10, 10])
    image.wrap(pbc=True)
    image.set_calculator(EMT())
    images.append(image)

### Define fingerprinting scheme

Here we demonstrate with the conventional Symmetry function. In our group, we have developed another fingerprinting scheme called GMP. Learn more about GMP at <https://arxiv.org/abs/2102.02390?utm_source=feedburner&utm_medium=feed&utm_campaign=Feed%253A+arxiv%252FQSXk+%2528ExcitingAds%2521+cs+updates+on+arXiv.org%2529>

A demo script for training with GMP is also included: 
https://github.com/ulissigroup/amptorch/tree/MCSH_paper1_lmdb/examples/GMP

In [17]:
Gs = {
    "default": {
        "G2": {
            "etas": np.logspace(np.log10(0.05), np.log10(5.0), num=4),
            "rs_s": [0],
        },
        "G4": {"etas": [0.005], "zetas": [1.0, 4.0], "gammas": [1.0, -1.0]},
        "cutoff": 6,
    },
}


### Define the configuration for trainer object

Detailed introductions and other specifications supported in trainer object can be found here under Section Usage/Configs: 
<https://github.com/ulissigroup/amptorch/tree/MCSH_paper1_lmdb>

In [18]:
config = {
    "model": {
        "get_forces": True,
        "num_layers": 3,
        "num_nodes": 5,
        "batchnorm": False,
    },
    "optim": {
        "force_coefficient": 0.04,
        "lr": 1e-2,
        "batch_size": 32,
        "epochs": 10,
        "loss": "mse",
        "metric": "mae",
        "gpus": 0,
    },
    "dataset": {
        "raw_data": images,
        "fp_params": Gs,
        "save_fps": True,
        "scaling": {"type": "normalize", "range": (0, 1)},
        "val_split": 0,
    },
    "cmd": {
        "debug": False,
        "run_dir": "./",
        "seed": 1,
        "identifier": "test",
        "verbose": True,
        # Weights and Biases used for logging - an account(free) is required
        "logger": False,
    },
}

### Hit go and train

In [19]:
torch.set_num_threads(1)
trainer = AtomsTrainer(config)
trainer.train()

Results saved to ./checkpoints/2021-06-25-14-15-53-test



Loading dataset: 100 images
Loading model: 963 parameters
Loading skorch trainer
  epoch    train_energy_mae    train_forces_mae    train_loss    cp     dur
-------  ------------------  ------------------  ------------  ----  ------
      1              0.6019              0.3522        0.4378     +  0.0160
      2              0.6040              0.3353        0.4280        0.0159
      3              0.5027              0.3252        0.3093     +  0.0160
      4              0.4639              0.3194        0.2469     +  0.0163
      5              0.4627              0.3167        0.2262     +  0.0159
      6              0.4268              0.3018        0.2792     +  0.0158
      7              0.3834              0.2814        0.1318     +  0.0159
      8              0.3514              0.2607        0.1458     +  0.0161
      9              0.2804              0.2340        0.0801     +  0.0159
     10              0.2119              0.2126        0.0960     +  0.0159
Traini

0.18897771835327148

### Use the neural network as an ase calculator to make predictions

In [20]:
predictions = trainer.predict(images)

# assess errors

true_energies = np.array([image.get_potential_energy() for image in images])
pred_energies = np.array(predictions["energy"])

print("Energy MSE:", np.mean((true_energies - pred_energies) ** 2))
print("Energy MAE:", np.mean(np.abs(true_energies - pred_energies)))

image.set_calculator(AMPtorch(trainer))
image.get_potential_energy()


Energy MSE: 0.07593011091772355
Energy MAE: 0.1824281114474392


11.364912033081055